# Coding Assignment 4 (Part II) 

## The Baum-Welch algorihtm

The Baum-Welch Algorihtm is the EM algorithm for HMM. You should prepare a function BW.onestep to perform the E-step and M-step, and then iteratively call that function in myBW.

In [20]:
# The Baum-Welch algorihtm

myBW = function(x, para, n.iter = 100){
  # Input:
  # x: T-by-1 observation sequence
  # para: initial parameter value
  # Output updated para value (A and B; we do not update w)

  for(i in 1:n.iter){
    para = BW.onestep(x, para)
  }
  return(para)
}

In [21]:
####### Your function BW.onestep, in which we operate the E-step and M-step for one iteration, should look as follows.

BW.onestep = function(x, para){
  # Input: 
  # x: T-by-1 observation sequence
  # para: mx, mz, and current para values for
  #    A: initial estimate for mz-by-mz transition matrix
  #    B: initial estimate for mz-by-mx emission matrix
  #    w: initial estimate for mz-by-1 initial distribution over Z_1
  # Output the updated parameters after one iteration
  # We DO NOT update the initial distribution w

  # Initialization step
  T = length(x)
  mz = para$mz
  mx = para$mx
  A = para$A
  B = para$B
  w = para$w
  alp = forward.prob(x, para)
  beta = backward.prob(x, para)
  
  myGamma = array(0, dim=c(mz, mz, T-1))
  #######################################
  ## YOUR CODE: 
  ## Compute gamma_t(i,j) P(Z[t] = i, Z[t+1]=j), 
  ## for t=1:T-1, i=1:mz, j=1:mz, 
  ## which are stored in an array, myGamma
  
  #######################################
  
  # M-step for parameter A
  #######################################
  ## YOUR CODE: 
  ## A = ....
  #######################################
  
  for(t in 1:T-1){
    denominator = ((alp[t,] %*% A) * B[,x[t+1]]) %*% matrix(beta[t+1,]) 
    for(s in 1:mz){
      numerator = alp[t,s] * A[s,] * B[,x[t+1]] * beta[t+1,]
      myGamma[s,,t]=numerator/as.vector(denominator)
    }
  }
  
  myGamma.all.t = rowSums(myGamma, dims = 2)
  A = myGamma.all.t/rowSums(myGamma.all.t)
  
  # M-step for parameter B
  #######################################
  ## YOUR CODE: 
  ## B = ....
  #######################################
  
  gamma = apply(myGamma, c(1, 3), sum)  
  gamma = cbind(gamma, colSums(myGamma[, , T-1]))
  for(l in 1:mx){
    B[, l] = rowSums(gamma[, which(x==l)])
  }
  B = B/rowSums(B)
  
  para$A = A
  para$B = B
  return(para)
}

Calculate forward and backward probabilities

In [22]:

# forward-backward probabilities

forward.prob = function(x, para){
  # Output the forward probability matrix alp 
  # alp: T by mz, (t, i) entry = P(x_{1:t}, Z_t = i)
  T = length(x)
  mz = para$mz
  A = para$A
  B = para$B
  w = para$w
  alp = matrix(0, T, mz)
  
  # fill in the first row of alp
  alp[1, ] = w * B[, x[1]]
  # Recursively compute the remaining rows of alp
  for(t in 2:T){
    tmp = alp[t-1, ] %*% A
    alp[t, ] = tmp * B[, x[t]]
  }
  return(alp)
}

backward.prob = function(x, para){
  # Output the backward probability matrix beta
  # beta: T by mz, (t, i) entry = P(x_{1:t}, Z_t = i)
  T = length(x)
  mz = para$mz
  A = para$A
  B = para$B
  w = para$w
  beta = matrix(1, T, mz)
  
  # The last row of beta is all 1.
  # Recursively compute the previous rows of beta
  for(t in (T-1):1){
    tmp = as.matrix(beta[t+1, ] * B[, x[t+1]])  # make tmp a column vector
    beta[t, ] = t(A %*% tmp)
  }
  return(beta)
}


## The Viterbi algorihtm

In [23]:
myViterbi = function(x, para){
  # Output: most likely sequence of Z (T-by-1)
  T = length(x)
  mz = para$mz
  A = para$A
  B = para$B
  w = para$w
  log.A = log(A)
  log.w = log(w)
  log.B = log(B)
  
  previous = matrix(0, T-1, mz)
  # Compute delta (in log-scale)
  delta = matrix(0, mz, T) 
  # fill in the first row of delta
  delta[, 1] = log.w + log.B[, x[1]]
  
  #######################################
  ## YOUR CODE: 
  ## Recursively compute the remaining rows of delta
  #######################################
  for(t in 2:T){
    for(s in 1:mz) {
      probs = delta[, t - 1] + log.A[, s] + log.B[s, x[t]]
      previous[t - 1, s] = which.max(probs)
      delta[s, t] = max(probs)
    }
  }
  
  
  # Compute the most prob sequence Z
  Z = rep(0, T)
  # start with the last entry of Z
  last_state=which.max(delta[,ncol(delta)])
  Z[1]=last_state
  
  #######################################
  ## YOUR CODE: 
  ## Recursively compute the remaining entries of Z
  #######################################

  j=2
  for(i in (T-1):1){
    Z[j]=previous[i,last_state] 
    last_state=previous[i,last_state] 
    j=j+1
  }
  
  Z[which(Z==1)]='A'
  Z[which(Z==2)]='B'
  
  Z=rev(Z)
  

  return(Z)
}

## Test function

### Result
Try your code on the data provided on Campuswire. You can (i) use the initial values specified below or (ii) use your own initial values. For the latter, remember to set the seed as the last four digits of your UIN.

In [24]:
# Test your function

data = scan("coding4_part2_data.txt")

mz = 2
mx = 3
ini.w = rep(1, mz); ini.w = ini.w / sum(ini.w)
ini.A = matrix(1, 2, 2); ini.A = ini.A / rowSums(ini.A)
ini.B = matrix(1:6, 2, 3); ini.B = ini.B / rowSums(ini.B)
ini.para = list(mz = 2, mx = 3, w = ini.w,
                A = ini.A, B = ini.B)

myout = myBW(data, ini.para, n.iter = 100)
myout.Z = myViterbi(data, myout)
myout.Z[myout.Z==1] = 'A'
myout.Z[myout.Z==2] = 'B'

### Result from HMM

In [25]:
library(HMM)
hmm0 =initHMM(c("A", "B"), c(1, 2, 3),
              startProbs = ini.w,
              transProbs = ini.A, 
              emissionProbs = ini.B)
Rout = baumWelch(hmm0, data, maxIterations=100, delta=1E-9, pseudoCount=0)
Rout.Z = viterbi(Rout$hmm, data)

### Compare two results

Compare estimates for transition prob matrix A

In [26]:
# Compare two results
options(digits=8)
options()$digits

myout$A
Rout$hmm$transProbs

[1] 8

0.49793938,0.50206062
0.44883431,0.55116569


,A,B
A,0.49793938,0.50206062
B,0.44883431,0.55116569


Compare estimates for emission prob matrix B

In [27]:
myout$B
Rout$hmm$emissionProbs

0.22159897,0.20266127,0.57573976
0.34175148,0.17866665,0.47958186


,1,2,3
A,0.22159897,0.20266127,0.57573976
B,0.34175148,0.17866665,0.47958186


Compare the most probable Z sequence.

In [28]:
cbind(Rout.Z, myout.Z)[c(1:10, 180:200), ]

Rout.Z,myout.Z
A,A
A,A
A,A
A,A
A,A
A,A
A,A
B,B
A,A
A,A


In [29]:
sum(Rout.Z != myout.Z)

[1] 0